In [3]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk #pip install nltk
from nltk.corpus import stopwords
#nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ben/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#Drive
test = pd.read_csv('/content/drive/My Drive/GitHub/TP2/test_original.csv')

In [4]:
#Local
test = pd.read_csv('test_original.csv')

In [5]:
test.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


## Keyword

In [6]:
test['keyword'] = test['keyword'].str.replace('%20', ' ')

## Location

In [7]:
def clean_location(location):
    if location is not np.nan:
        location = location.lower()
        location = re.sub("[^a-zñ ]", "", location)
        location = re.sub("ny[c*]", "new york city", location)
        location = re.sub("us[a*]", "united states of america", location)
        location = re.sub("uk", "united kingdom", location)
    
    return location

In [8]:
test['location'] = test['location'].apply(clean_location)

In [9]:
test['location'].unique()[::100]

array([nan, 'duo lane', 'coos bay or', 'liverpool england',
       'narbin city  ', 'wolverhampton low hill', 'hounslow london',
       'metro lampung  balikpapan', 'milton keynes ', 'android land',
       'northwest arkansas', 'harlem new york', 'portland',
       'seek for light today', 'louavul ky'], dtype=object)

## Text

In [10]:
english = 'abcdefghijklmnopqrstuvwxyz0123456789 '
text = {}
len_english = len(english)
len_test = len(test['text'])
for i in range(0, len_test):
    len_text = len(test['text'].iloc[i])
    for j in range(0, len_text):
        lower_case = test['text'].iloc[i].lower()
        special_character = []
        if lower_case[j] not in english:
            if lower_case[j] in text:
                special_character = text[lower_case[j]]
            special_character.append(i)
            text[lower_case[j]] = special_character

In [11]:
text.keys()

dict_keys(['#', ',', '.', "'", '?', '!', '-', ':', '/', '@', '\x89', 'û', 'ª', '\n', '_', '&', ';', 'å', '¨', '(', ')', '[', ']', 'ì', 'ò', '|', '~', 'ê', '$', 'ó', '*', 'ñ', '÷', '£', 'á', '©', '+', 'â', '¢', '¼', 'ï', '\x9d', '%', '¡', '=', 'è', 'ä', '^', 'à', '\\', '{', '}', '\x81'])

In [12]:
def show_sample():
    for key in text.keys():
        sample = text[key][0]
        print(test['text'].iloc[sample])
        
show_sample()

Heard about #earthquake is different cities, stay safe everyone.
Heard about #earthquake is different cities, stay safe everyone.
Heard about #earthquake is different cities, stay safe everyone.
We're shaking...It's an earthquake
They'd probably still show more life than Arsenal did yesterday, eh? EH?
Hey! How are you?
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU
@sunkxssedharry will you wear shorts for race ablaze ?
#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriage crisis sets Nigerian Twitter ablaze... http://t.co/CMghxBa2XI
#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriage crisis sets Nigerian Twitter ablaze... http://t.co/CMghxBa2XI
#PreviouslyOnDoyinTv: Toke MakinwaÛªs ma

In [13]:
def replace_text(text):
    text = text.replace('&amp', 'and')
    text = text.replace('amp;', 'and')
    text = text.replace('~', ' ')
    text = text.replace('Ûª', '\'')
    text = text.replace('ÛÒ', ' ')
    text = text.replace('ÛÓ', ' ')
    text = text.replace('&gt;', ' ')
    text = text.replace('&lt;', ' ')
    text = text.replace('ÛÏ', ' ')
    text = text.replace('Û', ' ')
    text = text.replace('\n', ' ')
    text = text.replace('å«', '\'')
    text = text.replace('åÊ', ' ')
    text = text.replace('åÇ', ' ')
    text = text.replace('Ì©', 'e')
    text = text.replace('Ì¤', 'c')
    text = text.replace('Ì¼', 'u')
    text = text.replace('Ì_', 'o')
    text = text.replace('`', '\'' )
    text = text.replace('\x89', '')
    text = text.replace('\x9d', '')
    
    return text

In [14]:
test['text'] = test['text'].apply(replace_text)

In [15]:
porter=nltk.PorterStemmer()
multiple_letters="adhijklnqruvwxyz"


def clean_text(text):
    text = text.lower()
    
    text = re.sub("(http|https)\S+", "link", text)
    text = re.sub(" can(\'*)t ", " cannot ", text)
    text = re.sub("n(\'*)t ", " not ", text)
    text = re.sub("\'s ", " is ", text)
    text = re.sub("\'re ", " are ", text)
    text = re.sub("\'ve ", " have ", text)
    text = re.sub("\'d ", " would ", text)
    text = re.sub("\'m ", " am ", text)

    for letter in multiple_letters:
        re_gex = letter + "(" + letter + "+)"
        text = re.sub(re_gex, letter, text)
    
    text = re.sub("\.|/|\+|-|÷|\*|:", " ", text)
    text = re.sub("( ' )|( # )", " ", text)
    text = re.sub("[^a-z 0-9]", "", text)
    #text = re.sub("[^a-z ]", "", text)
    
    split_text = re.split("(\d+)", text)
    text = " ".join(split_text)
        
    text = re.sub("[0-9]\d*", "number", text)
    
    token_text = nltk.word_tokenize(text)    
    new_text = []
    for word in token_text:
        if word not in stopwords.words('english'):
            new_text.append(porter.stem(word))
            new_text.append(" ")
    
    text = "".join(new_text)
    text = re.sub(" +", " ", text)
    
    return text

In [16]:
test['text'] = test['text'].apply(clean_text)

In [17]:
show_sample()

heard earthquak differ citi stay safe everyon 
heard earthquak differ citi stay safe everyon 
heard earthquak differ citi stay safe everyon 
shake earthquak 
would probabl stil show life arsen yesterday eh eh 
hey 
birmingham wholesal market ablaz bbc news fire break birmingham wholesal market link 
birmingham wholesal market ablaz bbc news fire break birmingham wholesal market link 
birmingham wholesal market ablaz bbc news fire break birmingham wholesal market link 
sunkxssedhari wil wear short race ablaz 
previouslyondoyintv toke makinwa mariag crisi set nigerian twitter ablaz link 
previouslyondoyintv toke makinwa mariag crisi set nigerian twitter ablaz link 
previouslyondoyintv toke makinwa mariag crisi set nigerian twitter ablaz link 
psa split person techi folow ablazeco burner folow ablaz 
psa split person techi folow ablazeco burner folow ablaz 
bewar world ablaz siera leon guap 
bewar world ablaz siera leon guap 
number car record zeroedg dual len car camera vehicl traffic dr

In [18]:
tf_idf_vec = TfidfVectorizer(use_idf=True, smooth_idf=False, ngram_range=(1,1), tokenizer=nltk.word_tokenize)
tf_idf_data = tf_idf_vec.fit_transform(test['text'])
col_names = tf_idf_vec.get_feature_names()

In [19]:
col_names

['ab',
 'aba',
 'abandon',
 'abba',
 'abbog',
 'abbott',
 'abbss',
 'abbswinston',
 'abbydphilip',
 'abc',
 'abcfmelissajoey',
 'abcnetwork',
 'abcnew',
 'abdul',
 'abe',
 'abela',
 'abh',
 'abid',
 'abil',
 'abl',
 'ablaz',
 'ablazeco',
 'abnormali',
 'abolish',
 'abomin',
 'aboooooortiooooon',
 'aborigin',
 'abort',
 'abound',
 'aboveandbeyond',
 'abriana',
 'absol',
 'absolut',
 'abstorm',
 'abt',
 'abus',
 'academi',
 'acapela',
 'accept',
 'access',
 'accid',
 'accidentali',
 'accionempresa',
 'accomplish',
 'accord',
 'accou',
 'account',
 'accur',
 'accuracyesport',
 'accus',
 'acdelco',
 'ace',
 'achiev',
 'achimota',
 'acid',
 'acoust',
 'acquisit',
 'acr',
 'acreativedc',
 'across',
 'act',
 'action',
 'activ',
 'activist',
 'actual',
 'actuali',
 'ad',
 'adam',
 'adamrubinespn',
 'adamtuss',
 'adani',
 'adapt',
 'adariu',
 'ade',
 'adict',
 'adilghumro',
 'adjust',
 'admin',
 'administ',
 'administr',
 'admir',
 'admit',
 'adolwyn',
 'adress',
 'adult',
 'adultress',
 'advan

In [20]:
encoded_text = {}
tf_idf_array = tf_idf_data.toarray()
len_tf_idf_array = len(tf_idf_array)
len_col_names = len(col_names)
for i in range(len_col_names):
    data = {}
    for j in range(len_tf_idf_array):
        if tf_idf_array[j][i] != 0:
            data[j] = tf_idf_array[j][i]
        
    encoded_text[col_names[i]] = data

In [21]:
text = pd.DataFrame(tf_idf_array, columns=col_names)

In [22]:
len_text = len(test['text'])

for i in range(len_text):
    values = []
    for word in test['text'].iloc[i].split():
#          if word in encoded_text:
        values.append(encoded_text[word][i])
    if values:
        text['sum'].iloc[i] = sum(values)
        text['min'].iloc[i] = min(values)
        text['max'].iloc[i] = max(values)

In [25]:
text.head(10)

,ab,aba,abandon,abba,abbog,abbott,abbss,abbswinston,abbydphilip,abc,...,zirngast,zix,zjwhitman,zombi,zone,zoui,zouma,zuma,zvch,zx
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Guardado del dataframe

In [ ]:
test.to_csv('test.csv', index=False)

In [26]:
text[['sum', 'max', 'min']].to_csv('test_text.csv', index=False)